In [22]:
import pandas as pd

import logging
import logging.config
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder

import ast

In [23]:
# CONECTAR A AMIGOCLOUD
data = open('../api_key/agrocittca.data','r')
password = data.readlines()[0]
data.close()

In [25]:
datos_cnx = ast.literal_eval(password)
datos_cnx

{'ssh_host': '167.99.184.211',
 'ssh_port': 22,
 'localhost': '127.0.0.1',
 'localport': 3306,
 'ssh_user': 'forge',
 'sql_username': 'forge',
 'sql_password': '21q5DVqO9tbCQjnQkNnO',
 'sql_main_database': 'guabira'}

In [28]:
mypkey = paramiko.RSAKey.from_private_key_file('../api_key/llave', password='micontrasena')

# ssh variables
ssh_host = datos_cnx['ssh_host']
ssh_port = datos_cnx['ssh_port']
localhost = datos_cnx['localhost']
localport = datos_cnx['localport']
ssh_user = datos_cnx['ssh_user']

# database variables
sql_username = datos_cnx['sql_username']
sql_password = datos_cnx['sql_password']
sql_main_database = datos_cnx['sql_main_database']

In [29]:
def dbQuery(q):
    """ access the database over the SSH tunnel and execute the query """
    logger = logging.getLogger()
    logger.debug("Connecting to reporting server...", end="")
    with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(localhost, 3306)) as tunnel:
        logger.debug("Connected")
        logger.debug("Connecting to database...")
        try:
            conn = pymysql.connect(
                host='127.0.0.1', 
                user=sql_username,
                passwd=sql_password,
                db=sql_main_database,
                port=tunnel.local_bind_port,
                cursorclass=pymysql.cursors.DictCursor)
            logger.debug("Connected")
            logger.debug("Executing query...")
            cursor = conn.cursor()
            cursor.execute(q)
            data = cursor.fetchall()
            logger.debug("Done")
        finally:
            conn.close()
    return data

In [40]:
# SELECCIONAR TODOS LOS USUARIOS CAÑEROS ACTIVOS
result = dbQuery('select codigo, name, apellido_paterno, apellido_materno, username from users where estado=\'activo\' and tipo = \'propietario\'')

In [41]:
df = pd.DataFrame(result)

In [42]:
df.shape

(34, 5)

In [43]:
df

,codigo,name,apellido_paterno,apellido_materno,username
0,2816,ALBERTO,DONOSO,BLANCO,adonoso
1,12478,HUGO,SERRATE,COLAMARINO,hserrate
2,41594,AGROPECUARIA CAMPO DULCE S.R.L.,AP,AP,campodulce
3,111,utea,RIVERO,DE AGUILERA,utea
4,75,NELSON MARIANO,AGUILERA,TARADELLES,nmaguilera
5,42075,LUIS ALFREDO,ONOFRE,RIOJA,laonofre
6,6457,RODOLFO,JORDAN,TONDELLY,rjordan
7,41534,FERNANDO WALTER,CAMPBELL,MEDINA,fwcampbell
8,10697,JAIME EDUARDO,PAZ,REA,jepaz
9,2549,MARCO ANTONIO,CARMAGO,REQUENA,macamargo
